# First project dataset export

In [ ]:
import labelbox
import os
import dotenv
import requests
import tqdm
import io
import pandas
import cv2
import matplotlib.pyplot
import PIL.Image
import json
import sklearn.model_selection
import yaml
import os
import numpy
import time
from contextlib import suppress
from xml.dom.minidom import parseString
from lxml.etree import Element, SubElement, tostring
%matplotlib inline

In [ ]:
HERE   = 'notebook'
ROOT   = os.getcwd().replace(HERE, "")
ENV    = 'env'
CONFIG = dotenv.dotenv_values(os.path.join(ROOT, ENV))
SOURCE = 'resource/caries/labelbox'

In [ ]:
classification = {
    "normal_crevice":{'label':0, "color":(0, 255, 0)},
    "caries":{'label':1, "color":(255, 0, 0)}
}

# Download data

Create the client and conect to **Labelbox** service.

In [ ]:
client  = labelbox.Client(api_key=CONFIG['API_KEY'])
project = client.get_project(CONFIG['PROJECT_ID'])
dataset = client.get_dataset(CONFIG['DATASET_ID'])

Export the image from **Labelbox** service.

In [ ]:
# def detect(image, size=50):

#     image = numpy.array(image.convert('L'))
#     (height, width) = image.shape
#     center = (int(width / 2.0), int(height / 2.0))
#     transform = numpy.fft.fft2(image)
#     shift = numpy.fft.fftshift(transform)
#     shift[center[1] - size:center[1] + size, center[0] - size:center[0] + size] = 0
#     shift = numpy.fft.ifftshift(shift)
#     frequency = numpy.fft.ifft2(shift)
#     value = 20 * numpy.log(1+numpy.abs(frequency))
#     score = numpy.mean(value)
#     return(score)

# def detect(image):
#
#     image = numpy.array(image.convert('L'))
#     score = cv2.Laplacian(image, cv2.CV_64F).var()
#     return(score)

In [ ]:
# def parse(annotation, image):
def parse(annotation):
    
    # size =  image.size
    category, top, left, height, width = [], [], [], [], []
    if(annotation!=[]):
        
        for _, a in enumerate(annotation):
            
            
            c, t, l, h, w = a['value'], a['bbox']['top'], a['bbox']['left'], a['bbox']['height'], a['bbox']['width']
            if(c=='carious'): c = 'caries'
            pass
            # if(h<50 or w<25):

            #     x = int(numpy.mean([l, l+w]))
            #     y = int(numpy.mean([t, t+h]))
            #     if(w<25): w = 2 * w
            #     if(h<50): h = 2 * h
            #     l = max(x-int(w / 2), 0)
            #     t = max(y-int(h / 2), 0)
            #     if(l + w > size[1]): w = size[1] - l
            #     if(t + h > size[0]): h = size[0] - t
            #     print(l, t, w, h)
            #     pass
            # if((h<50) or (w<50)):

            #     x = int(numpy.mean([l, l+w]))
            #     y = int(numpy.mean([t, t+h]))
            #     t = y - 30
            #     l = x - 30
            #     h = 60
            #     w = 60
            #     pass
            
            # single = image.crop((l, t, l+w, t+h))
            # s = detect(single, size=30)
            # if(s < 10): 
                
            #     print(s)
            #     continue

            category += [c]
            top += [t]
            left += [l]
            height += [h]
            width += [w]
            continue

        pass
    
    y = {"category":category, "left":left, "top":top, "width":width, "height":height}
    y = pandas.DataFrame(y)
    return(y)

In [ ]:
loop = list(project.labels())
for item in tqdm.tqdm(loop):
    
    data = dict()
    data['id'] = item.data_row().external_id.split('.')[0]
    data['image'] = PIL.Image.open(io.BytesIO(requests.get(item.data_row().row_data).content))
    data['annotation'] = json.loads(item.label)['objects']
    pass

    
    # data['annotation'] = parse(data['annotation'], data['image'])
    data['annotation'] = parse(data['annotation'])
    pass

    storage = dict()
    storage['image'] = os.path.join(ROOT, SOURCE, "jpg/")
    storage['annotation'] = os.path.join(ROOT, SOURCE, "txt/")
    os.makedirs(storage['image'], exist_ok=True)
    os.makedirs(storage['annotation'], exist_ok=True)
    pass

    path = dict()
    path['image'] = os.path.join(storage['image'], "{}.jpg".format(data['id']))
    path['annotation'] = os.path.join(storage['annotation'], "{}.txt".format(data['id']))
    pass

    data['image'].save(path['image'])
    data['annotation'].to_csv(path['annotation'], index=False, header=0, sep=" ")
    time.sleep(0.5)
    continue

Ground truth storage.

In [ ]:
def preview(file='671', show=False):
    
    image = os.path.join(ROOT, SOURCE, "jpg", "{}.jpg".format(file))
    annotation = os.path.join(ROOT, SOURCE, "txt", "{}.txt".format(file))
    pass
    
    picture = cv2.imread(image)

    with suppress(pandas.errors.EmptyDataError):
    
        annotation = pandas.read_csv(annotation, header=None, sep=' ')
        for _, item in  annotation.iterrows():

            start = item[1], item[2]
            end = item[1]+item[3], item[2]+item[4]
            color = classification.get(item[0])['color']
            cv2.rectangle(picture, start, end, color, 4)
            pass
        
        pass

    picture = PIL.Image.fromarray(picture)    
    if(show):

        matplotlib.pyplot.subplot(1,1,1)
        matplotlib.pyplot.axis('off')
        matplotlib.pyplot.imshow(picture)
        pass
    
    return(picture)

In [ ]:
loop = os.listdir(os.path.join(ROOT, SOURCE, 'jpg/'))
for i in loop:
    
    file = i.split('.')[0]
    picture = preview(file, False)
    storage = os.path.join(ROOT, 'resource/caries/', 'truth/')
    os.makedirs(storage, exist_ok=True)
    picture.save(os.path.join(storage, i))
    continue

In [ ]:
mode = 'jpg'
loop = os.listdir(os.path.join(ROOT, SOURCE, mode))
identification = [i.split('.')[0] for i in loop]
with open(os.path.join(ROOT, 'resource/caries/', 'identification.txt'), 'w') as paper:

    for i in identification: paper.write(i + '\n')
    pass

---

# Second project datset export 

In [1]:
import labelbox
import os
import dotenv
import requests
import tqdm
import io
import pandas
import cv2
import matplotlib.pyplot
import PIL.Image
import json
import sklearn.model_selection
import yaml
import os
import numpy
import time
from contextlib import suppress
from xml.dom.minidom import parseString
from lxml.etree import Element, SubElement, tostring
%matplotlib inline

In [2]:
HERE   = 'notebook'
ROOT   = os.getcwd().replace(HERE, "")
ENV    = 'env'
CONFIG = dotenv.dotenv_values(os.path.join(ROOT, ENV))
SOURCE = 'resource/adult-dentition/labelbox'

In [3]:
classification = {
    "臼齒":{'label':0, "color":(0, 255, 0)},
    "智齒":{'label':1, "color":(255, 0, 0)}
}

# Download data

Create the client and conect to **Labelbox** service.

In [4]:
client  = labelbox.Client(api_key=CONFIG['API_KEY'])
project = client.get_project('cl6k7sksn0xvf07wyaxkx1ekh')

In [5]:
def parse(annotation):
    
    category, top, left, height, width = [], [], [], [], []
    if(annotation!=[]):
        
        for _, a in enumerate(annotation):
            
            
            c, t, l, h, w = a['value'], a['bbox']['top'], a['bbox']['left'], a['bbox']['height'], a['bbox']['width']
            category += [c]
            top += [t]
            left += [l]
            height += [h]
            width += [w]
            continue

        pass
    
    y = {"category":category, "left":left, "top":top, "width":width, "height":height}
    y = pandas.DataFrame(y)
    return(y)

In [6]:
loop = list(project.labels())
for item in tqdm.tqdm(loop):
    
    data = dict()
    data['id'] = item.data_row().external_id.split('.')[0]
    data['image'] = PIL.Image.open(io.BytesIO(requests.get(item.data_row().row_data).content))
    data['annotation'] = json.loads(item.label)['objects']
    pass

    data['annotation'] = parse(data['annotation'])
    pass

    storage = dict()
    storage['image'] = os.path.join(ROOT, SOURCE, "jpg/")
    storage['annotation'] = os.path.join(ROOT, SOURCE, "txt/")
    os.makedirs(storage['image'], exist_ok=True)
    os.makedirs(storage['annotation'], exist_ok=True)
    pass

    path = dict()
    path['image'] = os.path.join(storage['image'], "{}.jpg".format(data['id']))
    path['annotation'] = os.path.join(storage['annotation'], "{}.txt".format(data['id']))
    pass

    data['image'].save(path['image'])
    data['annotation'].to_csv(path['annotation'], index=False, header=0, sep=" ")
    time.sleep(1)
    continue

100%|██████████| 719/719 [38:03<00:00,  3.18s/it]


Ground truth storage.

In [7]:
def preview(file='671', show=False):
    
    image = os.path.join(ROOT, SOURCE, "jpg", "{}.jpg".format(file))
    annotation = os.path.join(ROOT, SOURCE, "txt", "{}.txt".format(file))
    pass
    
    picture = cv2.imread(image)

    with suppress(pandas.errors.EmptyDataError):
    
        annotation = pandas.read_csv(annotation, header=None, sep=' ')
        for _, item in  annotation.iterrows():

            start = item[1], item[2]
            end = item[1]+item[3], item[2]+item[4]
            color = classification.get(item[0])['color']
            cv2.rectangle(picture, start, end, color, 4)
            pass
        
        pass

    picture = PIL.Image.fromarray(picture)    
    if(show):

        matplotlib.pyplot.subplot(1,1,1)
        matplotlib.pyplot.axis('off')
        matplotlib.pyplot.imshow(picture)
        pass
    
    return(picture)

In [8]:
loop = os.listdir(os.path.join(ROOT, SOURCE, 'jpg/'))
for i in loop:
    
    file = i.split('.')[0]
    picture = preview(file, False)
    storage = os.path.join(ROOT, 'resource/adult-dentition/', 'truth/')
    os.makedirs(storage, exist_ok=True)
    picture.save(os.path.join(storage, i))
    continue

In [ ]:
mode = 'jpg'
loop = os.listdir(os.path.join(ROOT, SOURCE, mode))
identification = [i.split('.')[0] for i in loop]
with open(os.path.join(ROOT, 'resource/adult-dentition/', 'identification.txt'), 'w') as paper:

    for i in identification: paper.write(i + '\n')
    pass

---

以下省略

Save box image.

In [ ]:
loop = pandas.read_csv(os.path.join(ROOT, 'resource/caries', 'identification.txt'), header=None)[0].tolist()
for i in loop:
    
    skip = False
    try:

        image = PIL.Image.open(os.path.join(ROOT, SOURCE, 'jpg', "{}.jpg".format(i)))
        annotation = pandas.read_csv(os.path.join(ROOT, SOURCE, 'txt', "{}.txt".format(i)), sep=" ", header=None)
        pass

    except:

        skip = True
        pass

    if(not skip):

        for number, item in annotation.iterrows():

            storage = os.path.join(ROOT, 'resource/caries/classification/{}'.format(item[0]))
            os.makedirs(storage, exist_ok=True)
            (x, y, w, h) = item[1:].tolist()
            thing = image.crop((x, y, x+w, y+h))
            thing.save(os.path.join(storage, '{}_{}.jpg'.format(i, number)))
            pass
        
        pass

    continue

---

針對 Box 擷取的特徵工程(skip)

In [ ]:
import labelbox
import os
import dotenv
import requests
import tqdm
import io
import pandas
import cv2
import matplotlib.pyplot
import PIL.Image
import json
import sklearn.model_selection
import yaml
import os
import numpy
import time
from contextlib import suppress
from xml.dom.minidom import parseString
from lxml.etree import Element, SubElement, tostring
%matplotlib inline

In [ ]:
HERE   = 'notebook'
ROOT   = os.getcwd().replace(HERE, "")
ENV    = 'env'
CONFIG = dotenv.dotenv_values(os.path.join(ROOT, ENV))
SOURCE = 'resource/carious/labelbox'

In [ ]:
loop = pandas.read_csv(os.path.join(ROOT, 'resource/carious', 'identification.txt'), header=None)[0].tolist()
for i in loop:
    
    skip = False
    try:

        image = PIL.Image.open(os.path.join(ROOT, SOURCE, 'jpg', "{}.jpg".format(i)))
        annotation = pandas.read_csv(os.path.join(ROOT, SOURCE, 'txt', "{}.txt".format(i)), sep=" ", header=None)
        pass

    except:

        skip = True
        pass

    if(not skip):

        for number, item in annotation.iterrows():

            storage = os.path.join(ROOT, 'resource/carious/classification/{}'.format(item[0]))
            os.makedirs(storage, exist_ok=True)
            (x, y, w, h) = item[1:].tolist()
            if(w<32): 

                x = int(x-(w*1.0))
                w = int(w+(w*1.0*2))
                pass

            if(h<32): 

                y = int(y-(h*0.5))
                h = int(h+(h*0.5*2))
                pass

            thing = image.crop((x, y, x+w, y+h))
            thing.save(os.path.join(storage, '{}_{}.jpg'.format(i, number)))
            if('{}_{}.jpg'.format(i, number)=="2C_1.jpg"): print(thing.size)
            pass
        
        pass

    continue